In [1]:
!pip install fasttext

In [2]:
!pip install gensim

In [3]:
!pip install --upgrade keras-nlp
!pip install --upgrade keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.8/571.8 kB 8.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 44.9 MB/s eta 0:00:0000:01
  Attempting uninstall: keras
    Found existing installation: keras 3.3.3
    Uninstalling keras-3.3.3:
      Successfully uninstalled keras-3.3.3
  Attempting uninstall: keras-nlp
    Found existing installation: keras-nlp 0.12.1
    Uninstalling keras-nlp-0.12.1:
      Successfully uninstalled keras-nlp-0.12.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.3 MB/s eta 0:00:0000:010:01
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency 

In [4]:
import numpy as np
import gensim.downloader
import tensorflow as tf
import re
from tensorflow.keras.losses import Loss
from tensorflow import keras
import matplotlib.pyplot as plt
import keras_cv
import numpy as np
import plotly.express as px
import pandas as pd
import keras_nlp
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint, EarlyStopping
from tensorflow.keras import layers
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import classification_report
keras.utils.set_random_seed(42)
import nltk
from nltk.corpus import stopwords

2024-07-02 13:32:59.588356: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-02 13:32:59.588479: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-02 13:32:59.714916: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
!kaggle datasets download -d naseralqaydeh/named-entity-recognition-ner-corpus

Dataset URL: https://www.kaggle.com/datasets/naseralqaydeh/named-entity-recognition-ner-corpus
License(s): DbCL-1.0
 97%|████████████████████████████████████▋ | 4.00M/4.14M [00:00<00:00, 5.94MB/s]
100%|██████████████████████████████████████| 4.14M/4.14M [00:00<00:00, 4.45MB/s]


In [6]:
!mkdir data

In [7]:
!unzip /kaggle/working/named-entity-recognition-ner-corpus.zip -d /kaggle/working/data

Archive:  /kaggle/working/named-entity-recognition-ner-corpus.zip
  inflating: /kaggle/working/data/ner.csv  


In [129]:
df=pd.read_csv("/kaggle/working/data/ner.csv", encoding='unicode_escape', on_bad_lines='skip')


In [21]:
df.head()

,Sentence #,Sentence,POS,Tag
0,Sentence: 1,Thousands of demonstrators have marched throug...,"['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...","['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '..."
1,Sentence: 2,Families of soldiers killed in the conflict jo...,"['NNS', 'IN', 'NNS', 'VBN', 'IN', 'DT', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,Sentence: 3,They marched from the Houses of Parliament to ...,"['PRP', 'VBD', 'IN', 'DT', 'NNS', 'IN', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,Sentence: 4,"Police put the number of marchers at 10,000 wh...","['NNS', 'VBD', 'DT', 'NN', 'IN', 'NNS', 'IN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,Sentence: 5,The protest comes on the eve of the annual con...,"['DT', 'NN', 'VBZ', 'IN', 'DT', 'NN', 'IN', 'D...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."


In [130]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47959 entries, 0 to 47958
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Sentence #  47959 non-null  object
 1   Sentence    47959 non-null  object
 2   POS         47959 non-null  object
 3   Tag         47959 non-null  object
dtypes: object(4)
memory usage: 1.5+ MB


In [23]:
fig = px.histogram(df, x=df['Sentence'].apply(lambda x : len(x.split())),nbins=20)
fig.update_layout(
    title="words",
    xaxis_title="words per pharse",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()

**Data preprocessing**

In [131]:
df['Sentence'] = df['Sentence'].apply(lambda x: x.lower().split())
df["Tag"]=df["Tag"].apply(lambda x : re.sub('[^\w-]',' ',x.lower()).split())
max_seq=30
df['condition'] = df['Sentence'].apply(lambda x: len(x)<max_seq)
df['important'] = df['Sentence'].apply(lambda x: set(x)!=1)

In [132]:
df.head()

,Sentence #,Sentence,POS,Tag,condition,important
0,Sentence: 1,"[thousands, of, demonstrators, have, marched, ...","['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...","[o, o, o, o, o, o, b-geo, o, o, o, o, o, b-geo...",True,True
1,Sentence: 2,"[families, of, soldiers, killed, in, the, conf...","['NNS', 'IN', 'NNS', 'VBN', 'IN', 'DT', 'NN', ...","[o, o, o, o, o, o, o, o, o, o, o, o, o, o, o, ...",False,True
2,Sentence: 3,"[they, marched, from, the, houses, of, parliam...","['PRP', 'VBD', 'IN', 'DT', 'NNS', 'IN', 'NN', ...","[o, o, o, o, o, o, o, o, o, o, o, b-geo, i-geo...",True,True
3,Sentence: 4,"[police, put, the, number, of, marchers, at, 1...","['NNS', 'VBD', 'DT', 'NN', 'IN', 'NNS', 'IN', ...","[o, o, o, o, o, o, o, o, o, o, o, o, o, o, o]",True,True
4,Sentence: 5,"[the, protest, comes, on, the, eve, of, the, a...","['DT', 'NN', 'VBZ', 'IN', 'DT', 'NN', 'IN', 'D...","[o, o, o, o, o, o, o, o, o, o, o, b-geo, o, o,...",True,True


In [26]:
fig = px.histogram(df, x=df['condition'] ,nbins=50)
fig.update_layout(
    title="textCln",
    xaxis_title="summerization  with number of words",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()

In [27]:
fig = px.histogram(df, x=df['important'] ,nbins=50)
fig.update_layout(
    title="textCln",
    xaxis_title="summerization  with number of words",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()

In [28]:
df=df[(df["condition"]==True)]

In [29]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz

--2024-07-02 13:34:36--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.35.7.82, 13.35.7.50, 13.35.7.128, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.35.7.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
Saving to: 'cc.en.300.bin.gz.1'

cc.en.300.bin.gz.1  100%[===================>]   4.19G   116MB/s    in 35s     

2024-07-02 13:35:11 (123 MB/s) - 'cc.en.300.bin.gz.1' saved [4503593528/4503593528]



In [30]:
!gunzip "/kaggle/working/cc.en.300.bin.gz"

In [113]:
import fasttext
import fasttext.util
#fasttext.util.download_model('en', if_exists='ignore')
ft = fasttext.load_model('/kaggle/working/cc.en.300.bin')
print(ft.get_dimension())
#fasttext.util.reduce_model(ft, 100)
#print(ft.get_dimension())

300


In [122]:
print(ft.get_dimension())


300


In [133]:
def createVocFreq(pharses,vocabulary=None,idx2word=None,freq=None):
    if(vocabulary==None):
        vocabulary={}
        freq={}
        idx2word={}
        idx=-1
    else:
        idx=len(idx2word)-1
        
    for pharse in pharses:
        for word in pharse:
            if word not in vocabulary:
                idx+=1
                vocabulary[word]=idx
                idx2word[idx]=word
                freq[word]=1
            else:
                freq[word]+=1
    return vocabulary,idx2word,freq
def createVecSpace(idx2word,model,vectors=None):
    if vectors==None:
        vectors=[]
    for i in range(len(idx2word)):
        vectors.append(model.get_word_vector(idx2word[i]))
    return vectors

In [134]:
vocabulary,idx2word,freq=createVocFreq([['<pad>']])
vocabulary,idx2word,freq=createVocFreq(df['Sentence'].values,vocabulary,idx2word,freq)

In [135]:
print(len(vocabulary))
print(len(idx2word))

31818
31818


In [136]:
vectors=createVecSpace(idx2word,ft)

In [137]:
ft.get_nearest_neighbors("friday")

[(0.8834149837493896, 'monday'),
 (0.8478385806083679, 'thursday'),
 (0.8460848927497864, 'saturday'),
 (0.8356624841690063, 'wednesday'),
 (0.820127546787262, 'tuesday'),
 (0.7851765155792236, 'sunday'),
 (0.7487204670906067, 'firday'),
 (0.7456310391426086, 'thrusday'),
 (0.7336512207984924, 'friday-'),
 (0.7224543690681458, 'friday.')]

In [138]:
vectors=np.array(vectors)
print(vectors.shape)

(31818, 300)


In [139]:
df.head()

,Sentence #,Sentence,POS,Tag,condition,important
0,Sentence: 1,"[thousands, of, demonstrators, have, marched, ...","['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...","[o, o, o, o, o, o, b-geo, o, o, o, o, o, b-geo...",True,True
1,Sentence: 2,"[families, of, soldiers, killed, in, the, conf...","['NNS', 'IN', 'NNS', 'VBN', 'IN', 'DT', 'NN', ...","[o, o, o, o, o, o, o, o, o, o, o, o, o, o, o, ...",False,True
2,Sentence: 3,"[they, marched, from, the, houses, of, parliam...","['PRP', 'VBD', 'IN', 'DT', 'NNS', 'IN', 'NN', ...","[o, o, o, o, o, o, o, o, o, o, o, b-geo, i-geo...",True,True
3,Sentence: 4,"[police, put, the, number, of, marchers, at, 1...","['NNS', 'VBD', 'DT', 'NN', 'IN', 'NNS', 'IN', ...","[o, o, o, o, o, o, o, o, o, o, o, o, o, o, o]",True,True
4,Sentence: 5,"[the, protest, comes, on, the, eve, of, the, a...","['DT', 'NN', 'VBZ', 'IN', 'DT', 'NN', 'IN', 'D...","[o, o, o, o, o, o, o, o, o, o, o, b-geo, o, o,...",True,True


In [140]:
classes={'o':0,'i-geo':1, 'i-nat':2, 'i-org':3, 'b-eve':4, 'b-gpe':5, 'b-per':6, 'i-tim':7, 'i-eve':8, 'b-org':9, 'i-gpe':10, 'b-tim':11, 'b-art':12, 'i-per':13, 'i-art':14, 'b-geo':15, 'b-nat':16}

In [141]:
df['Sentence']=df['Sentence'].apply(lambda x:list(map(lambda word:vocabulary[word], x)))
df['Tag']=df['Tag'].apply(lambda x:list(map(lambda l:classes[l], x)))

In [142]:
df.head()

,Sentence #,Sentence,POS,Tag,condition,important
0,Sentence: 1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...","[0, 0, 0, 0, 0, 0, 15, 0, 0, 0, 0, 0, 15, 0, 0...",True,True
1,Sentence: 2,"[23, 2, 24, 25, 12, 10, 26, 27, 10, 28, 29, 30...","['NNS', 'IN', 'NNS', 'VBN', 'IN', 'DT', 'NN', ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",False,True
2,Sentence: 3,"[43, 5, 19, 10, 44, 2, 45, 8, 46, 47, 12, 48, ...","['PRP', 'VBD', 'IN', 'DT', 'NNS', 'IN', 'NN', ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 1, 0]",True,True
3,Sentence: 4,"[50, 51, 10, 38, 2, 52, 53, 54, 55, 56, 57, 58...","['NNS', 'VBD', 'DT', 'NN', 'IN', 'NNS', 'IN', ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",True,True
4,Sentence: 5,"[10, 9, 61, 62, 10, 63, 2, 10, 64, 65, 2, 66, ...","['DT', 'NN', 'VBZ', 'IN', 'DT', 'NN', 'IN', 'D...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 0, 0, 9,...",True,True


In [143]:
data=df['Sentence'].values
labels=df['Tag'].values

In [144]:
print(data.shape)
print(labels.shape)

(47959,)
(47959,)


In [145]:
padded_data = tf.keras.utils.pad_sequences(data, padding="post",maxlen=max_seq)
padded_classes = tf.keras.utils.pad_sequences(labels ,padding="post",maxlen=max_seq)

In [146]:
print(padded_data.shape)
print(padded_classes.shape)

(47959, 30)
(47959, 30)


In [147]:
x_train,x_test,y_train,y_test=train_test_split(padded_data, padded_classes, test_size=0.15, random_state=42)

In [148]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(40765, 30)
(40765, 30)
(7194, 30)
(7194, 30)


In [149]:
x_train[0]

array([ 4474,  5500,  3883,   710,    10,  1285,    67, 11351,  5373,
        1016,   313,   114,    35,    46,  3817, 11170,  1405,  3131,
          91,  1690,    53,   279,   888,   315,    14,  2659,     1,
         979,    22,     0], dtype=int32)

**Model creation**

In [150]:
max_seq=max_seq
vec_size=ft.get_dimension()
#vocabulary,vectors

In [201]:
Input=layers.Input(shape=(max_seq,),name="Input_encoder")
embedding_layer = layers.Embedding(input_dim=len(vocabulary), output_dim=vec_size, weights=vectors,
                               trainable=False,name="encoder_embedding", mask_zero=True)
encoder_embedding=embedding_layer(Input)

position=keras_nlp.layers.PositionEmbedding(sequence_length=max_seq)
pos=position(encoder_embedding)

add=layers.Add()([encoder_embedding , pos])

encoder1=keras_nlp.layers.TransformerEncoder(512,64)
out_encoder1=encoder1(add)

encoder2=keras_nlp.layers.TransformerEncoder(512,64)
out_encoder2=encoder2(out_encoder1)

encoder3=keras_nlp.layers.TransformerEncoder(512,64)
out_encoder3=encoder3(out_encoder2)

encoder4=keras_nlp.layers.TransformerEncoder(512,64)
out_encoder4=encoder4(out_encoder3)

encoder5=keras_nlp.layers.TransformerEncoder(512,64)
out_encoder5=encoder5(out_encoder4)

dense1=layers.Dense(256,activation='relu')
dense1output=dense1(out_encoder5)

dense2=layers.Dense(len(classes),activation='softmax')
output=dense2(dense1output)

    
model = keras.Model(inputs=Input, outputs=output)
model.summary()

Model: "functional_24"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Input_encoder       │ (None, 30)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_embedding   │ (None, 30, 300)   │  9,545,400 │ Input_encoder[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ position_embedding… │ (None, 30, 300)   │      9,000 │ encoder_embeddin… │
│ (PositionEmbedding) │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_7 (Add)         │ (None, 30, 300)   │          0 │ encoder_embeddin… │
│                     │                   │            │ position_embeddi… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_encode… │ (None, 30, 300)   │    617,480 │ add_7[0][0]       │
│ (TransformerEncode… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_encode… │ (None, 30, 300)   │    617,480 │ transformer_enco… │
│ (TransformerEncode… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_encode… │ (None, 30, 300)   │    617,480 │ transformer_enco… │
│ (TransformerEncode… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_encode… │ (None, 30, 300)   │    617,480 │ transformer_enco… │
│ (TransformerEncode… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_encode… │ (None, 30, 300)   │    617,480 │ transformer_enco… │
│ (TransformerEncode… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_38 (Dense)    │ (None, 30, 256)   │     77,056 │ transformer_enco… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_39 (Dense)    │ (None, 30, 17)    │      4,369 │ dense_38[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 12,723,225 (48.54 MB)

 Trainable params: 3,177,825 (12.12 MB)

 Non-trainable params: 9,545,400 (36.41 MB)

In [73]:
pip install focal-loss

  Using cached keras-2.15.0-py3-none-any.whl.metadata (2.4 kB)
Using cached keras-2.15.0-py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
Note: you may need to restart the kernel to use updated packages.


In [109]:
import  focal_loss

In [215]:
early_stopping = EarlyStopping(monitor='val_accuracy',patience=5,verbose=0,mode='max',restore_best_weights=True)
callbacks = [early_stopping]
# Compile the model
#model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam', loss=focal_loss.SparseCategoricalFocalLoss(gamma=5), metrics=['accuracy'])

# Train the model
history=model.fit(x_train, y_train, epochs=20, batch_size=64  , validation_split = 0.2,    verbose = 1,callbacks = [early_stopping])

Epoch 1/20
  3/510 ━━━━━━━━━━━━━━━━━━━━ 28s 57ms/step - accuracy: 0.9692 - loss: 0.0266 

W0000 00:00:1719934304.186278     153 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


510/510 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.9642 - loss: 0.0253

W0000 00:00:1719934332.357630     153 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1719934334.877250     153 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


510/510 ━━━━━━━━━━━━━━━━━━━━ 55s 66ms/step - accuracy: 0.9642 - loss: 0.0253 - val_accuracy: 0.9623 - val_loss: 0.0362
Epoch 2/20


W0000 00:00:1719934337.863276     154 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


510/510 ━━━━━━━━━━━━━━━━━━━━ 21s 40ms/step - accuracy: 0.9638 - loss: 0.0253 - val_accuracy: 0.9614 - val_loss: 0.0351
Epoch 3/20
510/510 ━━━━━━━━━━━━━━━━━━━━ 21s 40ms/step - accuracy: 0.9634 - loss: 0.0250 - val_accuracy: 0.9617 - val_loss: 0.0346
Epoch 4/20
510/510 ━━━━━━━━━━━━━━━━━━━━ 21s 40ms/step - accuracy: 0.9636 - loss: 0.0242 - val_accuracy: 0.9618 - val_loss: 0.0347
Epoch 5/20
510/510 ━━━━━━━━━━━━━━━━━━━━ 21s 40ms/step - accuracy: 0.9644 - loss: 0.0224 - val_accuracy: 0.9611 - val_loss: 0.0345
Epoch 6/20
510/510 ━━━━━━━━━━━━━━━━━━━━ 21s 40ms/step - accuracy: 0.9642 - loss: 0.0218 - val_accuracy: 0.9592 - val_loss: 0.0360


In [206]:
y_pred=model.predict(x_test)

 33/225 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step

W0000 00:00:1719933964.836276     154 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


225/225 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step


In [212]:
print(classification_report(y_test.flatten(), np.argmax(y_pred,axis=-1).flatten()))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99    192716
           1       0.62      0.73      0.67      1070
           2       0.00      0.00      0.00         9
           3       0.69      0.42      0.52      2356
           4       0.64      0.15      0.25        46
           5       0.92      0.93      0.92      2243
           6       0.75      0.82      0.78      2398
           7       0.80      0.51      0.62       945
           8       0.38      0.07      0.12        41
           9       0.77      0.44      0.56      2799
          10       0.75      0.12      0.21        25
          11       0.86      0.83      0.85      2956
          12       0.00      0.00      0.00        59
          13       0.83      0.86      0.84      2460
          14       0.00      0.00      0.00        43
          15       0.79      0.88      0.83      5614
          16       0.41      0.23      0.29        40

    accuracy              